In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.autograd import Variable
import matplotlib.pyplot as plt
import sys
from random import shuffle




transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

cifar_train = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
cifar_valid = datasets.CIFAR10('./data', train=False, download=True, transform=transform)


ctrainset = torch.utils.data.DataLoader(cifar_train, batch_size=100, shuffle=True, num_workers=1)
ctestset = torch.utils.data.DataLoader(cifar_valid, batch_size=100, shuffle=True, num_workers=1)

dataiter = iter(ctrainset)
testiter = iter(ctestset)




class CNNAI(nn.Module):
    def __init__(self):
        super().__init__()
        self.Conv1 = nn.Conv2d(3, 9, 5, groups=3)
        self.Conv2 = nn.Conv2d(9, 50, 5)
        self.Conv3 = nn.Conv2d(50, 150, 5)
        self.MaxPool2 = nn.MaxPool2d(2)
        self.Relu1 = nn.ReLU()
        self.Relu2 = nn.ReLU()
        
        self.Lin1 = nn.Linear(9600, 960)
        self.Lin2 = nn.Linear(960, 96)
        self.Lin3 = nn.Linear(96, 30)
        self.Lin31 = nn.Linear(96, 48)
        self.BiLin1 = nn.Bilinear(30, 48, 20)
        self.Lin4 = nn.Linear(20, 10)
        
        self.Drop1 = nn.Dropout(p=.1)
        self.Drop2d = nn.Dropout2d(p=.1)
        
    def forward(self, x, train):
        x = self.Conv1(x)
        if train == True:
            x = self.Drop2d(x)
        x = self.Conv2(x)
        x = self.MaxPool2(x)
        x = self.Conv3(x)
        x = x.view(-1, 9600)
        x = self.Lin1(x)
        #print(x)
        if train == True:
            x = self.Drop1(x)
        x = self.Lin2(x)
        z = self.Lin3(x)
        y = self.Lin31(x)
        x = self.BiLin1(z, y)
        x = self.Lin4(x)
        return x
        
        
AI1 = CNNAI()
AI1 = AI1.cuda()
print(AI1)

OptimF1 = torch.optim.Rprop(AI1.parameters(), lr=0.0001)

LossF1 = nn.CrossEntropyLoss()


BigLoops = 100
CurrentIteration = 0
totalloss = 0.0
Indexes = []
train_loss = []
train_accuracy = []


for potato in range(BigLoops):
    NumTried = 0
    NumCorrect = 0
    CurrentIteration = 0
    shuffle(Indexes)
    AI1.train()
    for iteration, data in enumerate(ctrainset, 0) :
        CurrentIteration += 100
        pvalues, cclass = data
        pvalues = pvalues.cuda()
        cclass = cclass.cuda()
        OptimF1.zero_grad()
        guesses= AI1(pvalues, True)
        loss = LossF1(guesses, cclass)
        totalloss += loss.item()
        loss.backward()
        OptimF1.step()      
        for index in range(100):
            zzz, finalguess = torch.max(guesses[index], 0)
            NumCorrect += (finalguess == cclass[index]).sum()
        if iteration % 10 == 0:
            print("Images Processed:", iteration*100)
    NumTried += 50000 
        

    
    train_loss.append(totalloss/CurrentIteration)
    train_accuracy.append((100 * NumCorrect / 50000))
    
    totalloss = 0.0
    NumCorrect = 0
    
    print ('Epoch %d/%d, Loss: %.4f, Acc: %.4f' %(potato+1, BigLoops, train_loss[-1], train_accuracy[-1]))
    
    AI1.eval()
    
print("Done with training. Moving on to testing set.")

totaltestloss = 0
TestNumCorrect = 0
for iteration, data in enumerate(ctestset, 0):
    pvalues, cclass =  data
    pvalues = pvalues.cuda()
    cclass = cclass.cuda()
    guesses = AI1(pvalues, False)
    loss = LossF1(guesses, cclass)
    totaltestloss += loss.item()
    for index in range(100):
        _, finalguess = torch.max(guesses[index], 0)
        TestNumCorrect += (finalguess == cclass[index]).sum()
        
print("Testing done. AI got %d/%d images correct."%(TestNumCorrect, 10000))    


Files already downloaded and verified
Files already downloaded and verified
CNNAI(
  (Conv1): Conv2d(3, 9, kernel_size=(5, 5), stride=(1, 1), groups=3)
  (Conv2): Conv2d(9, 50, kernel_size=(5, 5), stride=(1, 1))
  (Conv3): Conv2d(50, 150, kernel_size=(5, 5), stride=(1, 1))
  (MaxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Relu1): ReLU()
  (Relu2): ReLU()
  (Lin1): Linear(in_features=9600, out_features=960, bias=True)
  (Lin2): Linear(in_features=960, out_features=96, bias=True)
  (Lin3): Linear(in_features=96, out_features=30, bias=True)
  (Lin31): Linear(in_features=96, out_features=48, bias=True)
  (BiLin1): Bilinear(in1_features=30, in2_features=48, out_features=20, bias=True)
  (Lin4): Linear(in_features=20, out_features=10, bias=True)
  (Drop1): Dropout(p=0.1)
  (Drop2d): Dropout2d(p=0.1)
)
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed

Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Image

Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 13/100, Loss: 0.0161, Acc: 44.0000
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Proce

Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 20/100, Loss: 0.0155, Acc: 46.0000
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Proce

Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000

...

Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 73/100, Loss: 0.0131, Acc: 54.0000
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Proce

Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 80/100, Loss: 0.0129, Acc: 55.0000
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Proce

Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 87/100, Loss: 0.0126, Acc: 56.0000
Images Processed: 0
Ima

Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 93/100, Loss: 0.0125, Acc: 56.0000
Images Processed: 0
Images Processed: 1000
Images Processed: 2000
Images Processed: 3000
Images Processed: 4000
Images Processed: 5000
Images Processed: 6000
Images Processed: 7000
Images Processed: 8000
Images Processed: 9000
Images Processed: 10000
Images Processed: 11000
Images Processed: 12000
Images Processed: 13000
Images Processed: 14000
Images Processed: 15000
Images Processed: 16000
Images Processed: 17000
Images Processed: 18000
Images Processed: 19000
Images Processed: 20000
Images Processed: 21000
Images Processed: 22000
Images Processed: 23000
Images Processed: 24000
Images Processed: 25000
Images Processed: 26000
Images Processed: 27000
Images Processed: 28000
Images Processed: 29000
Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Proce

Images Processed: 30000
Images Processed: 31000
Images Processed: 32000
Images Processed: 33000
Images Processed: 34000
Images Processed: 35000
Images Processed: 36000
Images Processed: 37000
Images Processed: 38000
Images Processed: 39000
Images Processed: 40000
Images Processed: 41000
Images Processed: 42000
Images Processed: 43000
Images Processed: 44000
Images Processed: 45000
Images Processed: 46000
Images Processed: 47000
Images Processed: 48000
Images Processed: 49000
Epoch 100/100, Loss: 0.0123, Acc: 57.0000
DONE! MWAHAHAHA
Testing done. AI got 5743/10000 images correct.
